In [1]:
import argparse
import os
from pathlib import Path
from typing import Optional
import yt_dlp
from rich import print as rich_print

In [2]:
def ensure_directory(path: str) -> None:
    """Create directory if it does not exist."""
    os.makedirs(path, exist_ok=True)

In [3]:
def download_youtube_to_mp4(
    url: str,
    output_dir: str,
    filename_stem: Optional[str] = None,
) -> str:
    """Download a YouTube video and save as mp4 in output_dir.

    Tries to prefer mp4 formats; falls back to best available and remuxes to mp4 when possible.
    Returns the path to the resulting file if it can be determined, otherwise the directory.
    """

    ensure_directory(output_dir)

    # Build output template
    if filename_stem:
        # Ensure .mp4 extension in final name
        outtmpl = os.path.join(output_dir, f"{filename_stem}.%(ext)s")
    else:
        # Use YouTube title; yt-dlp will set the extension
        outtmpl = os.path.join(output_dir, "%(title)s.%(ext)s")

    # Prefer progressive MP4; otherwise merge best streams and remux to mp4 (requires ffmpeg)
    format_selector = "best[ext=mp4]/bv*[ext=mp4]+ba[ext=m4a]/b/bv*+ba"

    # Track final file path via hooks because postprocessors may change extension
    final_path_container: dict[str, Path] = {}

    def _hook(d: dict) -> None:
        if d.get("status") == "finished":
            filename = d.get("filename")
            if filename:
                final_path_container["path"] = Path(filename)

    ydl_opts: dict = {
        "format": format_selector,
        "outtmpl": outtmpl,
        "noplaylist": True,
        "merge_output_format": "mp4",
        "postprocessors": [
            {"key": "FFmpegVideoRemuxer", "preferedformat": "mp4"},
        ],
        "progress_hooks": [_hook],
        "quiet": False,
    }

    rich_print(f"[bold green]Downloading[/] → {url}")
    rich_print(f"[bold]Output dir[/]: {output_dir}")

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    # Try to resolve the final mp4 path
    if "path" in final_path_container:
        candidate = final_path_container["path"]
    else:
        # Fallback: construct from prepared filename
        with yt_dlp.YoutubeDL({"outtmpl": outtmpl}) as ydl:
            prepared = Path(ydl.prepare_filename(info))
        candidate = prepared

    # Normalize to .mp4 if remuxed
    mp4_candidate = candidate.with_suffix(".mp4")
    if mp4_candidate.exists():
        rich_print(f"[bold green]Saved[/]: {mp4_candidate}")
        return mp4_candidate

    # If not mp4, return whatever was produced
    if candidate.exists():
        rich_print(f"[yellow]Saved (non-mp4)[/]: {candidate}")
        return candidate

    rich_print("[red]Could not determine final file path, but download likely completed.[/]")
    return output_dir

## Final_Destination (official PG 13 trailer)

In [5]:
url = "https://www.youtube.com/watch?v=UWMzKXsY9A4"
output_dir = "data/YT_download/"
name = "Final_Destination"

result_path = download_youtube_to_mp4(
    url=url,
    output_dir=output_dir,
    filename_stem=name,
)
rich_print(f"[bold]Result[/]: {result_path}")

Downloading → https://www.youtube.com/watch?v=UWMzKXsY9A4

Output dir: data/YT_download/

[youtube] Extracting URL: https://www.youtube.com/watch?v=UWMzKXsY9A4
[youtube] UWMzKXsY9A4: Downloading webpage
[youtube] UWMzKXsY9A4: Downloading tv client config
[youtube] UWMzKXsY9A4: Downloading tv player API JSON
[youtube] UWMzKXsY9A4: Downloading ios player API JSON


[youtube] UWMzKXsY9A4: Downloading m3u8 information
[info] UWMzKXsY9A4: Downloading 1 format(s): 18
[download] Destination: data/YT_download/Final_Destination.mp4
[download] 100% of    9.40MiB in 00:00:00 at 13.84MiB/s  
[VideoRemuxer] Not remuxing media file "data/YT_download/Final_Destination.mp4"; already is in target format mp4


Saved: data/YT_download/Final_Destination.mp4

Result: data/YT_download/Final_Destination.mp4

## Final_Destination_All_Deaths (fanmade M18/R21 scenes)

In [ ]:
url = "https://www.youtube.com/watch?v=tLnGQxjShB4"
output_dir = "data/YT_download/"
name = "Final_Destination_All_Deaths"

result_path = download_youtube_to_mp4(
    url=url,
    output_dir=output_dir,
    filename_stem=name,
)
rich_print(f"[bold]Result[/]: {result_path}")